# Distance traveled and amount of interactions

In [1]:
import os, sys
import toml
from src import settings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.utils import fileio

CONFIG_PATH = os.path.join(settings.CONFIG_DIR, "main.toml")
with open(CONFIG_PATH, "r") as file:
    config = toml.load(file)
    
TIME_WINDOW=config["TIME_WINDOW"]

INPUT_PATH = os.path.join(settings.RESULTS_DIR, "distances_traveled")
all_treatments = fileio.load_multiple_folders(INPUT_PATH)
all_treatments = {key: value for key, value in all_treatments.items() if key in config["TREATMENTS"]}

treatment_dataframes = []
for treatment_name, treatment_path in all_treatments.items():
    all_groups = fileio.load_files_from_folder(treatment_path)
    group_dataframes = []
    for group_name, df_path in all_groups.items():
        df = pd.read_csv(df_path, index_col=0)  
        df['Snapshot'] = (df.index / config["FPS"] / TIME_WINDOW).astype(int) + 1
        df['Group'] = group_name.replace(".csv","")
        group_dataframes.append(df)

    df = pd.concat(group_dataframes)
    df.set_index('Snapshot', inplace=True)
    df.set_index('Group', append=True, inplace=True)
    df['Treatment'] = treatment_name
    df.set_index('Treatment', append=True, inplace=True)
    df = df.reorder_levels(['Treatment', 'Group', 'Snapshot'])
  
    treatment_dataframes.append(df)
    
data = pd.concat(treatment_dataframes)
data = data.groupby(['Treatment', 'Group']).sum()
data = data.stack(level=0).reset_index(level=[0, 1])
data.reset_index(inplace=True)
data.columns = ['Fly_name', 'Treatment', 'Group', 'Distance']
data = data[['Treatment', 'Group', 'Fly_name', 'Distance']]

In [43]:
INPUT_PATH = "/home/milky/drosophila-SNA/data/processed/1_0_find_interactions/"
all_treatments = fileio.load_multiple_folders(INPUT_PATH)
all_treatments = {key: value for key, value in all_treatments.items() if key in config["TREATMENTS"]}

treatment_dataframes = []
for treatment_name, treatment_path in all_treatments.items():
    all_groups = fileio.load_files_from_folder(treatment_path)
    group_dataframes = []
    for group_name, df_path in all_groups.items():
        df = pd.read_csv(df_path, index_col=0)  
        df = df["node_1"]
        counts = df.value_counts()
        df['Group'] = group_name.replace(".csv","")

        group_dataframes.append(df)
        sys.exit()

SystemExit: 

/home/milky/drosophila-SNA/.env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [46]:

counts.columns = ["coounts"]
counts

node_1
fly11    327
fly4     323
fly12    315
fly10    302
fly1     280
fly8     280
fly9     277
fly5     259
fly6     240
fly3     233
fly7     227
fly2     210
Name: count, dtype: int64